## Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import time
import copy
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn.functional import softmax
import torch.nn.functional as F
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, \
    AdaptiveAvgPool2d, Sequential, Module
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.models import mobilenet_v2

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
!unzip -q {'drive/MyDrive/dataset.zip'} -d {'./'}

### Face Extractor

In [4]:
# Function to extract face with OpenCV
def extract_face(image, show=False):
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        return Image.fromarray(image_cv)
    x1, y1, w1, h1 = faces[0]

    face = Image.fromarray(image).crop((x1, y1, x1 + w1, y1 + h1))
    if show:
        plt.imshow(face)
        plt.show()
    return face

## Deep Learning Model

### Get the Dataset Ready

In [5]:
# prepare data
class FASDatasetDeep(Dataset):
    def __init__(self, df, transforms=None, ft_width=28, ft_height=28):
        self.df = df
        self.transforms = transforms
        self.ft_width = ft_width
        self.ft_height = ft_height

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['Filepath']
        bbox_path = img_path[:-4] + '_BB.txt'

        sample = extract_face(img_path)
        target = df_train.iloc[idx][43]

        # Generate the FT picture of the sample
        ft_sample = self.generate_FT(sample)

        if self.transforms is not None:
            try:
                sample = self.transforms(sample)
            except Exception as err:
                print('Error Occured: %s' % err, img_path)

        assert sample is not None

        ft_sample = cv2.resize(ft_sample, (self.ft_width, self.ft_height))
        ft_sample = torch.from_numpy(ft_sample).float()
        ft_sample = torch.unsqueeze(ft_sample, 0)

        return sample, ft_sample, target

    def generate_FT(self, image):
        image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
        f = np.fft.fft2(image)
        fshift = np.fft.fftshift(f)
        fimg = np.log(np.abs(fshift)+1)
        maxx = -1
        minn = 100000
        for i in range(len(fimg)):
            if maxx < max(fimg[i]):
                maxx = max(fimg[i])
            if minn > min(fimg[i]):
                minn = min(fimg[i])
        fimg = (fimg - minn+1) / (maxx - minn+1)
        return fimg

In [26]:
# transformations
transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

### Model Structure

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using \'{}\' device'.format(device))

using 'cuda' device


In [8]:
keep_dict = {'1.8M': [32, 32, 103, 103, 64, 13, 13, 64, 26, 26,
                      64, 13, 13, 64, 52, 52, 64, 231, 231, 128,
                      154, 154, 128, 52, 52, 128, 26, 26, 128, 52,
                      52, 128, 26, 26, 128, 26, 26, 128, 308, 308,
                      128, 26, 26, 128, 26, 26, 128, 512, 512],

             '1.8M_': [32, 32, 103, 103, 64, 13, 13, 64, 13, 13, 64, 13,
                       13, 64, 13, 13, 64, 231, 231, 128, 231, 231, 128, 52,
                       52, 128, 26, 26, 128, 77, 77, 128, 26, 26, 128, 26, 26,
                       128, 308, 308, 128, 26, 26, 128, 26, 26, 128, 512, 512]
             }

class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_c, kernel_size=kernel, groups=groups,
                           stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel,
                           groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class Depth_Wise(Module):
     def __init__(self, c1, c2, c3, residual=False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        c1_in, c1_out = c1
        c2_in, c2_out = c2
        c3_in, c3_out = c3
        self.conv = Conv_block(c1_in, out_c=c1_out, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(c2_in, c2_out, groups=c2_in, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(c3_in, c3_out, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual

     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output


class Residual(Module):
    def __init__(self, c1, c2, c3, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for i in range(num_block):
            c1_tuple = c1[i]
            c2_tuple = c2[i]
            c3_tuple = c3[i]
            modules.append(Depth_Wise(c1_tuple, c2_tuple, c3_tuple, residual=True,
                                      kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)

    def forward(self, x):
        return self.model(x)


# Define the Fourier Transform Generator
class FTGenerator(nn.Module):
    def __init__(self, in_channels=128, out_channels=1):
        super(FTGenerator, self).__init__()

        self.ft = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.ft(x)

# Modify the MobileNetV2 model
class MobileNetV2_FTDeep(nn.Module):
    def __init__(self, num_classes=2, freeze_backbone=True):
        super(MobileNetV2_FTDeep, self).__init__()
        keep = keep_dict['1.8M_']
        self.conv1 = Conv_block(3, keep[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(keep[0], keep[1], kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=keep[1])

        c1 = [(keep[1], keep[2])]
        c2 = [(keep[2], keep[3])]
        c3 = [(keep[3], keep[4])]

        self.conv_23 = Depth_Wise(c1[0], c2[0], c3[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=keep[3])

        c1 = [(keep[4], keep[5]), (keep[7], keep[8]), (keep[10], keep[11]), (keep[13], keep[14])]
        c2 = [(keep[5], keep[6]), (keep[8], keep[9]), (keep[11], keep[12]), (keep[14], keep[15])]
        c3 = [(keep[6], keep[7]), (keep[9], keep[10]), (keep[12], keep[13]), (keep[15], keep[16])]

        self.conv_3 = Residual(c1, c2, c3, num_block=4, groups=keep[4], kernel=(3, 3), stride=(1, 1), padding=(1, 1))

        c1 = [(keep[16], keep[17])]
        c2 = [(keep[17], keep[18])]
        c3 = [(keep[18], keep[19])]

        self.conv_34 = Depth_Wise(c1[0], c2[0], c3[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=keep[19])

        c1 = [(keep[19], keep[20]), (keep[22], keep[23]), (keep[25], keep[26]), (keep[28], keep[29]),
              (keep[31], keep[32]), (keep[34], keep[35])]
        c2 = [(keep[20], keep[21]), (keep[23], keep[24]), (keep[26], keep[27]), (keep[29], keep[30]),
              (keep[32], keep[33]), (keep[35], keep[36])]
        c3 = [(keep[21], keep[22]), (keep[24], keep[25]), (keep[27], keep[28]), (keep[30], keep[31]),
              (keep[33], keep[34]), (keep[36], keep[37])]

        self.conv_4 = Residual(c1, c2, c3, num_block=6, groups=keep[19], kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.model = mobilenet_v2(pretrained=True)

        # Freeze the backbone layers
        if freeze_backbone:
            for param in self.model.parameters():
                param.requires_grad = False

        # Unfreeze the last two layers
        for param in self.model.features[-2:].parameters():
            param.requires_grad = True

        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
        self.FTGenerator = FTGenerator(in_channels=128)

    def forward(self, x):
        features = self.model.features(x)
        # x1 = self.model.avgpool(x)
        # Global average pooling
        x1 = nn.functional.adaptive_avg_pool2d(features, (1, 1))
        x1 = torch.flatten(x1, 1)
        cls_output = self.model.classifier(x1)

        if self.training:
            x = self.conv1(x)
            x = self.conv2_dw(x)
            x = self.conv_23(x)
            x = self.conv_3(x)
            x = self.conv_34(x)
            x = self.conv_4(x)
            ft_output = self.FTGenerator(x)
            return cls_output, ft_output
        else:
            return cls_output

In [10]:
# Load the saved model weights and evaluate
pretrain_weights = 'drive/MyDrive/fas_mobilenetv2_v4.pth'
DeepModel = MobileNetV2_FTDeep(num_classes=2)
DeepModel.load_state_dict(torch.load(pretrain_weights)['model_state_dict'])

<All keys matched successfully>

In [11]:
DeepModel.to(device)

MobileNetV2_FTDeep(
  (conv1): Conv_block(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=32)
  )
  (conv2_dw): Conv_block(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=32)
  )
  (conv_23): Depth_Wise(
    (conv): Conv_block(
      (conv): Conv2d(32, 103, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(103, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=103)
    )
    (conv_dw): Conv_block(
      (conv): Conv2d(103, 103, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=103, bias=False)
      (bn): BatchNorm2d(103, eps=1e-05, momentum=0.1, affine=True, track_runni

## Feature Engineering Model

### Extract Engineered Features

In [12]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt


def extract_lbp_features(ImagePath):

    image = cv2.resize(cv2.cvtColor(ImagePath , cv2.COLOR_BGR2GRAY) , (255 , 255))
    # Compute the Local Binary Pattern (LBP)
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')

    return np.expand_dims(lbp , axis = 2)


def extract_fourier_features(ImagePath):

    image = cv2.resize(cv2.cvtColor(ImagePath , cv2.COLOR_BGR2GRAY) , (255 , 255))
     # Compute the 2D Fast Fourier Transform (FFT)
    f = np.fft.fft2(image)

    # Shift the zero frequency component to the center
    fshift = np.fft.fftshift(f)

    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(fshift)

    # Normalize the magnitude spectrum
    magnitude_spectrum = np.log(1 + magnitude_spectrum)  # Use log scaling for better visualization


    return np.expand_dims(magnitude_spectrum , axis = 2)


def extract_hsv_features(ImagePath, bins=8):


    image = cv2.resize(ImagePath , (255 , 255))
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    return hsv_image


def extract_features(ImagePath):

    ImagePath = np.array(ImagePath)
    lbp = extract_lbp_features(ImagePath)
    fourier = extract_fourier_features(ImagePath)
    hsv = extract_hsv_features(ImagePath)

    return np.concatenate((lbp , fourier , hsv) , axis = 2)

### Get the Dataset Ready

In [13]:
# prepare data
class FASDataset(Dataset):
    def __init__(self, df, transforms=None, ft_width=32, ft_height=32):
        self.df = df
        self.transforms = transforms
        self.ft_width = ft_width
        self.ft_height = ft_height

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['Filepath']
        bbox_path = img_path[:-4] + '_BB.txt'

        face = extract_face(img_path)
        sample = np.transpose(extract_features(face) , (2 , 1 , 0))
        target = df.iloc[idx][43]

        # Generate the FT picture of the sample
        ft_sample = self.generate_FT(face)

        if self.transforms is not None:
            try:
                sample = self.transforms(face)
                sample = np.transpose(extract_features(sample) , (2 , 1 , 0))
            except Exception as err:
                print('Error Occured: %s' % err, img_path)

        assert sample is not None

        ft_sample = cv2.resize(ft_sample, (self.ft_width, self.ft_height))
        ft_sample = torch.from_numpy(ft_sample).float()
        ft_sample = torch.unsqueeze(ft_sample, 0)

        return sample, ft_sample, target

    def generate_FT(self, image):
        image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
        f = np.fft.fft2(image)
        fshift = np.fft.fftshift(f)
        fimg = np.log(np.abs(fshift)+1)
        maxx = -1
        minn = 100000
        for i in range(len(fimg)):
            if maxx < max(fimg[i]):
                maxx = max(fimg[i])
            if minn > min(fimg[i]):
                minn = min(fimg[i])
        fimg = (fimg - minn+1) / (maxx - minn+1)
        return fimg

### Model Structure

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using \'{}\' device'.format(device))

using 'cuda' device


In [15]:
keep_dict = {'1.8M': [32, 32, 103, 103, 64, 13, 13, 64, 26, 26,
                      64, 13, 13, 64, 52, 52, 64, 231, 231, 128,
                      154, 154, 128, 52, 52, 128, 26, 26, 128, 52,
                      52, 128, 26, 26, 128, 26, 26, 128, 308, 308,
                      128, 26, 26, 128, 26, 26, 128, 512, 512],

             '1.8M_': [32, 32, 103, 103, 64, 13, 13, 64, 13, 13, 64, 13,
                       13, 64, 13, 13, 64, 231, 231, 128, 231, 231, 128, 52,
                       52, 128, 26, 26, 128, 77, 77, 128, 26, 26, 128, 26, 26,
                       128, 308, 308, 128, 26, 26, 128, 26, 26, 128, 512, 512]
             }

class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_c, kernel_size=kernel, groups=groups,
                           stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel,
                           groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class Depth_Wise(Module):
     def __init__(self, c1, c2, c3, residual=False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        c1_in, c1_out = c1
        c2_in, c2_out = c2
        c3_in, c3_out = c3
        self.conv = Conv_block(c1_in, out_c=c1_out, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(c2_in, c2_out, groups=c2_in, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(c3_in, c3_out, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual

     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output


class Residual(Module):
    def __init__(self, c1, c2, c3, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for i in range(num_block):
            c1_tuple = c1[i]
            c2_tuple = c2[i]
            c3_tuple = c3[i]
            modules.append(Depth_Wise(c1_tuple, c2_tuple, c3_tuple, residual=True,
                                      kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)

    def forward(self, x):
        return self.model(x)


# Define the Fourier Transform Generator
class FTGenerator(nn.Module):
    def __init__(self, in_channels=128, out_channels=1):
        super(FTGenerator, self).__init__()

        self.ft = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.ft(x)

# Modify the MobileNetV2 model
class MobileNetV2_FT(nn.Module):
    def __init__(self, num_classes=2, freeze_backbone=True):
        super(MobileNetV2_FT, self).__init__()
        keep = keep_dict['1.8M_']
        self.conv1 = Conv_block(5, keep[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(keep[0], keep[1], kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=keep[1])

        c1 = [(keep[1], keep[2])]
        c2 = [(keep[2], keep[3])]
        c3 = [(keep[3], keep[4])]

        self.conv_23 = Depth_Wise(c1[0], c2[0], c3[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=keep[3])

        c1 = [(keep[4], keep[5]), (keep[7], keep[8]), (keep[10], keep[11]), (keep[13], keep[14])]
        c2 = [(keep[5], keep[6]), (keep[8], keep[9]), (keep[11], keep[12]), (keep[14], keep[15])]
        c3 = [(keep[6], keep[7]), (keep[9], keep[10]), (keep[12], keep[13]), (keep[15], keep[16])]

        self.conv_3 = Residual(c1, c2, c3, num_block=4, groups=keep[4], kernel=(3, 3), stride=(1, 1), padding=(1, 1))

        c1 = [(keep[16], keep[17])]
        c2 = [(keep[17], keep[18])]
        c3 = [(keep[18], keep[19])]

        self.conv_34 = Depth_Wise(c1[0], c2[0], c3[0], kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=keep[19])

        c1 = [(keep[19], keep[20]), (keep[22], keep[23]), (keep[25], keep[26]), (keep[28], keep[29]),
              (keep[31], keep[32]), (keep[34], keep[35])]
        c2 = [(keep[20], keep[21]), (keep[23], keep[24]), (keep[26], keep[27]), (keep[29], keep[30]),
              (keep[32], keep[33]), (keep[35], keep[36])]
        c3 = [(keep[21], keep[22]), (keep[24], keep[25]), (keep[27], keep[28]), (keep[30], keep[31]),
              (keep[33], keep[34]), (keep[36], keep[37])]

        self.conv_4 = Residual(c1, c2, c3, num_block=6, groups=keep[19], kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.model = mobilenet_v2(pretrained=True)
        original_conv = self.model.features[0][0]
        new_conv = nn.Conv2d(5, original_conv.out_channels, kernel_size=original_conv.kernel_size,
                             stride=original_conv.stride, padding=original_conv.padding, bias=original_conv.bias)

        # Copy the weights from the original conv layer to the new conv layer
        with torch.no_grad():
            new_conv.weight[:, :3, :, :] = original_conv.weight
            new_conv.weight[:, 3:, :, :] = original_conv.weight.mean(dim=1, keepdim=True)

        # Replace the original conv layer with the new conv layer in the model
        self.model.features[0][0] = new_conv

        # Freeze the backbone layers
        if freeze_backbone:
            for param in self.model.parameters():
                param.requires_grad = False

        # Unfreeze the last two layers
        for param in self.model.features[-2:].parameters():
            param.requires_grad = True

        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
        self.FTGenerator = FTGenerator(in_channels=128)

    def forward(self, x):
        features = self.model.features(x)
        x1 = nn.functional.adaptive_avg_pool2d(features, (1, 1))
        x1 = torch.flatten(x1, 1)
        cls_output = self.model.classifier(x1)

        if self.training:
            x = self.conv1(x)
            x = self.conv2_dw(x)
            x = self.conv_23(x)
            x = self.conv_3(x)
            x = self.conv_34(x)
            x = self.conv_4(x)
            ft_output = self.FTGenerator(x)
            return cls_output , ft_output
        else:
            return cls_output

In [16]:
# Load the saved model weights and evaluate
pretrain_weights = 'drive/MyDrive/fe_fas_mobilenetv2_v2.pth'
model = MobileNetV2_FT(num_classes=2)
model.load_state_dict(torch.load(pretrain_weights)['model_state_dict'])

<All keys matched successfully>

In [17]:
model.to(device)

MobileNetV2_FT(
  (conv1): Conv_block(
    (conv): Conv2d(5, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=32)
  )
  (conv2_dw): Conv_block(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=32)
  )
  (conv_23): Depth_Wise(
    (conv): Conv_block(
      (conv): Conv2d(32, 103, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(103, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=103)
    )
    (conv_dw): Conv_block(
      (conv): Conv2d(103, 103, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=103, bias=False)
      (bn): BatchNorm2d(103, eps=1e-05, momentum=0.1, affine=True, track_running_s

### Predict Crop Image Function

In [18]:
from torchvision.transforms.functional import to_pil_image
from PIL import Image, ImageDraw
from torchvision import transforms

def crop_predict(image_path, model, transform,isfeature = False, device = device ):
    model.eval()

    face = extract_face(image_path)
    if isfeature:
      input_tensor = torch.from_numpy(np.transpose(extract_features(face) , (2 , 1 , 0))).float().unsqueeze(0).to(device)
    else :
      input_tensor = transform(face).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)

    return  probs

### Predict the Whole Input Image

In [19]:
from torchvision.transforms.functional import to_pil_image
from PIL import Image, ImageDraw
from torchvision import transforms

def full_predict(image , model,transform,isfeature = False, device=device):
    model.eval()

    if isfeature:
      input_tensor = torch.from_numpy(np.transpose(extract_features(image) , (2 , 1 , 0))).float().unsqueeze(0).to(device)
    else:
      input_tensor = transform(Image.fromarray(image)).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)

    return  probs

### Predict the Fourier Transform of the Image

In [20]:
from torchvision import transforms as tf

def generate_FT(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    fimg = np.log(np.abs(fshift)+1)
    maxx = -1
    minn = 100000
    for i in range(len(fimg)):
        if maxx < max(fimg[i]):
            maxx = max(fimg[i])
        if minn > min(fimg[i]):
            minn = min(fimg[i])
    fimg = (fimg - minn+1) / (maxx - minn+1)
    return fimg

# Function for prediction with Fourier Transform
def predict_fourier(image , model,transform, isfeature = False, device=device):
    model.eval()

    ft_sample = generate_FT(image)
    ft_sample = cv2.resize(ft_sample, (224, 224))
    ft_sample = torch.from_numpy(ft_sample).float()
    ft_sample = torch.unsqueeze(ft_sample, 0)
    if isfeature:
      ft_sample = ft_sample.repeat(5, 1, 1)
    else:
      ft_sample = ft_sample.repeat(3, 1, 1)
    input_tensor = ft_sample.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)

        probs = torch.nn.functional.softmax(output, dim=1)

    return  probs

## Predict Input Video

In [21]:
def make_prediction(img, model , transform , isfeature = False):

    crop_prob = crop_predict(img , model , transform , isfeature)
    full_prob = full_predict(img , model , transform , isfeature)
    fourier_prob = predict_fourier(img , model , transform , isfeature)

    return  crop_prob , full_prob , fourier_prob

def PredictVideo(model , videopath , isfeature = False):
    probs = []
    crop_probs = []
    fourier_probs = []

    vid_capture = cv2.VideoCapture(videopath)


    frame_width = int(vid_capture.get(3))
    frame_height = int(vid_capture.get(4))
    frame_size = (frame_width, frame_height)
    print('Frame size  :', frame_size)

    if not vid_capture.isOpened():
        print("Error opening a video stream")
    # Reading fps and frame rate
    else:
        fps = vid_capture.get(5)
        print('Frame rate  : ', fps, 'FPS')
        if fps == 0:
            fps = 24

    while vid_capture.isOpened():
        ret, frame = vid_capture.read()
        if ret:
            crop_prob , full_prob , fourier_prob = make_prediction(frame, model , transforms['test'] , isfeature)
            probs.append(full_prob)
            crop_probs.append(crop_prob)
            fourier_probs.append(fourier_prob)

            torch.cuda.empty_cache()
        else:
            print("Streaming is Off")
            break

    vid_capture.release()

    probs = torch.stack(probs)
    probs = probs.mean(dim=0)

    crop_probs = torch.stack(crop_probs)
    crop_probs = crop_probs.mean(dim=0)

    fourier_probs = torch.stack(fourier_probs)
    fourier_probs = fourier_probs.mean(dim=0)

    return probs , crop_probs , fourier_probs

In [24]:
import os

# Function to read directory paths from a text file
def read_directory_paths(file_path):
    with open(file_path, 'r') as file:
        directory_paths = [line.strip() for line in file.readlines()]
    return directory_paths

directories = read_directory_paths('/content/drive/MyDrive/dataset.txt')

If you encounter the error message 'module['test'] not subscripbtable, you might consider running the cell dedicated to 'transforms' implementation again, and then this error would be gone!

In [27]:
import csv

header = ['filename' , 'liveness_score' , 'liveness_score_crop' , 'liveness_score_frequency']

with open('predictions_feature.csv' , 'w' , newline='') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(header)

with open('predictions_deep.csv' , 'w' , newline='') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(header)

for directory in directories:
  with open('predictions_feature.csv' , 'a' , newline='') as csvfile:
    csvwriter = csv.DictWriter(csvfile, fieldnames=header)
    probs , crop_probs , fourier_probs = PredictVideo(model , directory , isfeature=True)
    data = {
        'filename' : directory.split('/')[-1],
        'liveness_score' : float(probs[0][1]) ,
        'liveness_score_crop' : float(crop_probs[0][1]) ,
        'liveness_score_frequency' : float(fourier_probs[0][1])
    }
    csvwriter.writerow(data)

  with open('predictions_deep.csv' , 'a' , newline='') as csvfile:
    csvwriter = csv.DictWriter(csvfile, fieldnames=header)
    probs , crop_probs , fourier_probs = PredictVideo(DeepModel , directory)
    data = {
        'filename' : directory.split('/')[-1],
        'liveness_score' : float(probs[0][1]) ,
        'liveness_score_crop' : float(crop_probs[0][1]) ,
        'liveness_score_frequency' : float(fourier_probs[0][1])
    }
    csvwriter.writerow(data)

Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (1280, 720)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (1280, 720)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (464, 848)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (464, 848)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (464, 848)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (464, 848)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (720, 1280)
Frame rate  :  30.0 FPS
Streaming is Off
Frame size  : (464, 848)
Frame rate  :  30.0 FPS
Streaming is Off


In [28]:
deep_pred = pd.read_csv('/content/predictions_deep.csv')
deep_pred

,filename,liveness_score,liveness_score_crop,liveness_score_frequency
0,spoof1.mp4,0.639763,0.729876,0.767381
1,spoof2.mp4,0.760516,0.734344,0.758207
2,spoof3.mp4,0.664728,0.782748,0.777733
3,spoof4.mp4,0.798873,0.857166,0.767925
4,spoof5.mp4,0.677845,0.748057,0.785354
5,spoof6.mp4,0.730612,0.736595,0.796380
6,non-spoof1.mp4,0.603222,0.665796,0.829088
7,non-spoof2.mp4,0.707813,0.748223,0.787779
8,non-spoof3.mp4,0.718835,0.681930,0.770095
9,non-spoof4.mp4,0.704911,0.662527,0.784996


In [29]:
fe_pred = pd.read_csv('/content/predictions_feature.csv')
fe_pred

,filename,liveness_score,liveness_score_crop,liveness_score_frequency
0,spoof1.mp4,0.998902,0.537227,1.0
1,spoof2.mp4,0.999723,0.999999,1.0
2,spoof3.mp4,0.956085,0.900189,1.0
3,spoof4.mp4,0.991445,0.869822,1.0
4,spoof5.mp4,0.687150,0.964872,1.0
5,spoof6.mp4,0.868686,0.894257,1.0
6,non-spoof1.mp4,0.644829,0.790365,1.0
7,non-spoof2.mp4,0.999993,0.533375,1.0
8,non-spoof3.mp4,0.842644,0.997862,1.0
9,non-spoof4.mp4,0.972649,0.222489,1.0
